In [1]:
# install necessary libraries (assuming the initial block has been run)
# !pip install scikit-learn
# !pip install pandas
# !pip install nltk

# download necessary nltk resources (assuming the initial block has been run)
# import nltk
# nltk.download('punkt')
# nltk.download('punkt_tab')
# nltk.download('wordnet')
# nltk.download('stopwords')

print("NLTK Resources downloaded successfully!")

# Import necessary libraries
import pandas as pd
import numpy as np
import nltk
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# 🌟 替换：导入随机森林分类器 (Random Forest Classifier)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from joblib import dump,load

# Import NLTK
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

# Initialize NLTK resources
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Placeholder: Load the actual dataset. Ensure it has 'text' (user query) and 'intent' (label) columns
df = pd.read_csv('dataset.csv')

# shuffle the data for robust splitting
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df.head())

def preprocess_text(text):
    # 1. Convert to Lowercase
    text = text.lower()
    
    # 2. Remove Punctuation and Special Characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # 3. Tokenization
    tokens = word_tokenize(text)
    
    # 4. Stopword Removal
    tokens = [word for word in tokens if word not in stop_words]
    
    # 5. Lemmatization (Key Enhancement)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Rejoin tokens into a single string
    return ' '.join(tokens)

# Apply the new preprocessing function to the text column
df['cleaned_text'] = df['text'].apply(preprocess_text)
print("--- Preprocessing Complete (with NLTK Lemmatization) ---")
print(df[['text', 'cleaned_text']].head())

# Prepare the cleaned text and intents for the model training section
X = df['cleaned_text']
y = df['intent']

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the text data to create the feature matrix X
X = vectorizer.fit_transform(X)
y = df['intent']

print(f"Feature matrix X shape: {X.shape}")
print(f"Labels y shape: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    test_size=0.2,
    random_state=42,
    stratify=y if len(df['intent'].unique()) > 1 else None, # 仅在有多个类别时分层
)

print(f"Train set size:{X_train.shape[0]} samples")
print(f"Test set size:{X_test.shape[0]} samples")

# 🌟 替换：实例化随机森林 (Random Forest) 模型
# n_estimators: 森林中树的数量（越多越好，但计算成本越高）
# criterion: 衡量分割质量的函数
# random_state: 确保结果可复现
rf_model = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=42)

# 训练模型
rf_model.fit(X_train, y_train)

# Make predictions on the test set
pred = rf_model.predict(X_test)

print("--- Random Forest Model Evaluation ---")
print("Accuracy:", accuracy_score(y_test, pred))
print("\nClassification Report:\n")
if len(y_test.unique()) > 1:
    print(classification_report(y_test, pred, zero_division=0))
else:
    print("Classification Report skipped: Only one class in test set.")

# 🌟 替换：保存训练好的 Random Forest 模型和 Vectorizer
dump(rf_model, 'random_forest_intent_model.joblib')
dump(vectorizer, 'tfidf_vectorizerRF.joblib')
print("Model and Vectorizer saved using joblib.")

# Predefined fixed responses (Retrieval System)
print("\n--- Loading Responses from JSON ---")
try:
    with open('response.json', 'r', encoding='utf-8') as f:
        responses = json.load(f)
    print(f"Successfully loaded {len(responses)} intent-response pairs.")
except FileNotFoundError:
    print("Error: response.json file not found.")
    responses = {}

# 🌟 替换：更新函数名
def chatbot_reply_rf(user_input, model, vectorizer, responses):
    # 1. Preprocessing
    # ⚠️ 关键：这里必须使用完整的预处理函数，与训练保持一致
    user_input_cleaned = preprocess_text(user_input)
    
    # 2. Feature Extraction: Transform the input using the fitted vectorizer
    vector = vectorizer.transform([user_input_cleaned])

    # 3. Intent Prediction
    intent = model.predict(vector)[0]

    # 4. Retrieval (Check for unknown intent/fallback)
    # If the predicted intent exists in the dictionary, return the specific response
    # Otherwise, return a fallback message
    return responses.get(intent, f"Sorry, I predicted the intent '{intent}', but I don't have a specific response for that yet. Please rephrase your question.")

# Test the chatbot function
print("\n --- Random Forest Chatbot Test ---")
test_input = "What time can I check in?"
predicted_response = chatbot_reply_rf(test_input, rf_model, vectorizer, responses)
print(f"User Input: {test_input}")
print(f"Chatbot Reply: {predicted_response}")

NLTK Resources downloaded successfully!
                           text              intent
0               Zero tolerance?  ask_smoking_policy
1            Is the hotel open?    ask_availability
2  Are there fines for smoking?  ask_smoking_policy
3            Cancel reservation    ask_cancellation
4            Traveling with dog      ask_pet_policy
--- Preprocessing Complete (with NLTK Lemmatization) ---
                           text        cleaned_text
0               Zero tolerance?      zero tolerance
1            Is the hotel open?          hotel open
2  Are there fines for smoking?        fine smoking
3            Cancel reservation  cancel reservation
4            Traveling with dog       traveling dog
Feature matrix X shape: (2001, 974)
Labels y shape: (2001,)
Train set size:1600 samples
Test set size:401 samples
--- Random Forest Model Evaluation ---
Accuracy: 0.6932668329177057

Classification Report:

                        precision    recall  f1-score   support

  ask_a